In [1]:
#===============================================================================================================
# AI_HUB(https://aihub.or.kr/) 에 '대규모 웹데이터 기반 한국어 말뭉치 데이터'(json파일) 중에서
# ***라벨링된 파일를 파싱해서, 2줄짜리 txt 말뭉치 만드는 예제
# => 대규모 웹데이터 기반 한국어 말뭉치 데이터 
# : https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624
#
# 1. 위 URL에서 대규모 웹데이터 기반 한국어 말뭉치 데이터  다운로드
# 2. 압축 풀기 
# => unzip abc.zip -d <경로> # 특정 디렉토리에 압축 해제
# 
#==============================================================================================================
import torch
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import json
import os
import re
import sys
from typing import Dict, List, Optional
sys.path.append("..")
from myutils import getListOfFiles

In [2]:
# 폴더에 있는 파일 목록 얻기(*반드시 /로 끝나야함)
inpdirectory = '../../data11/ai_hub/vl1/test/'
files = getListOfFiles(inpdirectory)

print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

*file_count: 10, file_list:['../../data11/ai_hub/vl1/test/BWSC217000050253.json', '../../data11/ai_hub/vl1/test/BWHE217000028346.json', '../../data11/ai_hub/vl1/test/BWEC217000009721.json', '../../data11/ai_hub/vl1/test/BWED217000013574.json', '../../data11/ai_hub/vl1/test/BWAC217000002828.json']


In [3]:
min_text_len = 10        # MIN_SETENCE_LEN 이하인 문장은 추가 안함
max_text_len = 150       # MAX 문장 길이 이상인 경우에는 .으로 구분해서 나눔
SEP_STRING = ';$;'           # 2문장 구분자 => 문장A+[SET_STR]+문장B
SEP_TYPE = 2                 # 어떻게 한줄에 문장을 나눌지?(0=문단으로 나눔, 1=1줄로 나눔, 2=2줄로 나눔)
remove_pattern_list = ['@(이메일)', '<저작권자', '전재-재배포', '전재 및 재배포 금지>','무단 전재 및 재배포 금지']    # 문장에 remove_list에 들어간 경우 추가 안함 

# 출력 파일 열기 
outfilepath = '../../data11/ai_hub/vl1/vl1-2줄-test.txt'
f1 = open(outfilepath, 'wt', encoding='utf-8')
        
# SEP_TYPE 범위를 벗어나면 error 발생!
if SEP_TYPE < 0 or SEP_TYPE > 2:
    raise Exception("SEP_TYPE 에러 (범위:0~2)")
    
#================================================================================    
# 입력 문장 목록을 가지고 패턴 검사 및 긴 문장 .로 분할해서 문장 전처리하는 함수
# - inputlist = 문장 list
# - remove_pattern_list = 해당 패턴 있는 문장은 제거
# - min_len = 문장 최소길이(해당길이보다 작은 문장은 제거)
# - max_len = 해당 길이보다 긴 문장은 .(마침표) 문장을 분할함.
#================================================================================ 
def check_sentectlist(inputlist: List,       
                      remove_pattern_list: List, 
                      min_len: int=10, max_len: int=200):
    
    outlist = [] 
    for sentence in inputlist:
        text = sentence['sentence']
        tlen = len(text)
        
        if tlen > min_len:
               
            # remove 패턴인 경우에는 추가안함
            isremove = 0
            for pattern in remove_pattern_list:
                result = text.find(pattern)
                if result > 0:
                    isremove = 1
                    break
                              
            if isremove == 0:
                # 긴 문장인 경우에는 . 기준으로 나눠서 문장 만듬
                if tlen > max_len:
                    for text1 in text.split('.'):
                        if text1 != '' and len(text1) > min_len:
                            outlist.append(text1)
                else:
                    outlist.append(text)
    
    return outlist

#==================================================================================
# 2문장을 -> 1문장으로 합치기 
# - inputlist = 문장 list
# - sepstr = 문장1과 2 사이에 구분자
#==================================================================================
def line2(inputlist: List, 
          sepstr):
    
    start = 0
    outlist = []
    inputlist_len = len(inputlist)
    for idx in range(inputlist_len):
        # 2문장을 'sepstr' 기준으로 해서 1나의 문장으로. 만듬
        outlist.append(inputlist[start].strip() + sepstr + inputlist[start+1].strip())
        start += 2
                    
        if start >= inputlist_len-1:
            break  
    
    return outlist

In [4]:
   
# 해당 json 파일들을 불러옴
for idx, file in enumerate(tqdm(files)):
    if ".json" not in file:  #.json 파일이 아니면 합치지 않음
        continue
        
    # json 파일 로딩 => [named_entity] entry만 불러옴
    json_data = json.load(open(file, "r", encoding="utf-8"))['named_entity']

    for entry in json_data:
        
        sentences = []
        
        # title sentnece 추가
        title = entry['title']
        outlist = check_sentectlist(title, remove_pattern_list, min_text_len, max_text_len)
        for text in outlist:
            sentences.append(text)

        # content sentence 추가
        content = entry['content']
        outlist = check_sentectlist(content, remove_pattern_list, min_text_len, max_text_len)
        for text in outlist:
            sentences.append(text)
        
        
        # 문장이 2개 이상인 경우에만 처리 
        num_sentences = len(sentences)
        if num_sentences >= 2:
            
            if SEP_TYPE == 1:# 1줄씩 출력
                f1.write("\n".join(sentences))
                
            elif SEP_TYPE == 2: #2줄씩 출력
                data = line2(sentences, SEP_STRING)
                for data1 in data:
                    f1.write(data1+'\n')
                    
            elif SEP_TYPE == 0: # 문단단위로 출력
                f1.write(SEP_STRING.join(sentences))             
                f1.write('\n')
                                    
# 출력파일 닫기
f1.close() 


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
# out 생성된 파일 읽어오기(한줄씩 읽어오기)
with open(outfilepath, 'r', encoding='UTF8') as f:
     lines = f.readlines()
        
print(len(lines))

7127


In [6]:
print(lines[0:10])

["청호나이스, '먹는물·씻는물 안심' 정수기로 프리미엄시장 공략;$;청호나이스가 2가지 정수 시스템을 이용해 먹는 물과 씻는 물 모두 (이름)는 하이브리드 얼음정수기 ‘도도’를 출시했다.\n", '먹는 물에는 자사의 가장 뛰어난 필터 기술을 적용하고 식재료를 닦는 물에도 직수 필터 기술을 적용해 정수기 본질을 지킨 프리미엄 제품이라는 주장이다..;$;청호나이스는 환경오염과 생활 건강에 대한 소비자들 관심이 늘어나는 만큼 프리미엄 전략을 유지하며 직수 방식이나 다른 보급형 정수기를 쓰고 있는 가정을 고객으로 적극 끌어들일 계획이다.\n', '해외시장에도 활발히 나서 매년 국내외 제품 매출을 2배 수준으로 키운다는 포부다..;$;청호나이스는 9일 서울시 중구 소재 서울 웨스턴(이름)텔에서 하이브리드 얼음정수기 ‘도도’ 출시 발표회를 열었다..\n', '(이름) 청호나이스 대표는 최근 정수기들은 디자인, 가격 경쟁력만 강조하고 있지만 도도는 좋은 물이라는 정수기 본질에 집중했다고 강조했다..;$;이 대표는 “최근 정수기는 얼마나 작고 예쁜지 또 얼마나 저렴하게 렌탈할 수 있는지 등 디자인, 가격 측면만 강조돼 안타깝다”며 “청호나이스는 정수기 본질이 뭔지 고민했고 이런 고민 속에 도도를 출시했다”고 말했다..\n', '■ 목적따라 2가지 정수물 제공.;$;도도의 가장 큰 특징은 2가지 정수 시스템이다.\n', '먹는 물과 쌀, 채소 등 식재료 세척에 쓰는 물을 함께 제공한다.;$;두 시스템 목적이 다른 만큼 적용 기술도 차이가 있다..\n', '음용수는 청호나이스 주력 정수 방식인 ro 멤브레인 필터와 포스트 카본 필터(역삼투압 방식)를 거쳐 정수기 코크에서 나온다.;$;ro 멤브레인 필터는 0.0001미크론(μ) 기공 크기의 초정밀 분리막으로 미세 플라스틱뿐만 아니라 중금속, 박테리아, 유기화학물질, 불소, 질산성 질소 등 유해 이온성 물질까지 제거한다..\n', '청호나이스는 최고 품질의 ro 멤브레인 필터를 확보하기 위해 미국 화학업체 다우 케미칼(dow c